<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design with a Resfile and FastRelax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

# Working With Antibodies
Keywords: Antibody, RosettaAntibody, RosettaAntibodyDesign, RAbD


## Overview
If you are here, you probably understand a thing or two about antibodies.  Lets do a quick review anyway.
<img src="./Media/antibody/antibody_structure_overview.png" width=1000>

## IgG
Here is the full IgG that has two `Fabs` or antibody fragments.  Each Fab has a variable (`Fv`) and constant (`Fc`) region.  In Rosetta, we are mostly concerned with Fv - where the binding region actually resides. The molecule that an antibody binds to is called an `antigen`. 
<img src="./Media/antibody/antibody_structure_IgG.png" width=1000>

## CDRs
An antibody (Fv) is made up of 6 Complementarity Determining Regions, or CDRs. The H3 CDR is the most variable CDR, but L1, L3, and H2 also take on many different shapes and lengths and in many cases, are just as critical to binding. 

The CDR region - specifically what is bound to the antigen is called the `paratope`, while the region on the antigen is called the `epitope`.
<img src="./Media/antibody/antibody_structure_cdrs.png" width=750>

In [ ]:
# Notebook setup
!pip install pyrosettacolabsetup
import pyrosettacolabsetup
pyrosettacolabsetup.setup()
print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

**Make sure you are in the directory with the pdb files:**

`cd google_drive/My\ Drive/student-notebooks/`

## Imports

Before we begin, we must import some specific machinery from Rosetta.  Much of these tools are automatically imported when we do `from pyrosetta import *`, however, some are not. You should get into the habit of importing everything you need.  This will get you comfortable with the organization of Rosetta and make it easier to find tools that are beyond the scope of these workshops.

In [29]:
#Python
from __future__ import print_function
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *

#Core Includes
from rosetta.core.kinematics import MoveMap
from rosetta.core.kinematics import FoldTree
from rosetta.core.pack.task import TaskFactory
from rosetta.core.pack.task import operation
from rosetta.core.simple_metrics import metrics
from rosetta.core.select import residue_selector as selections
from rosetta.core import select
from rosetta.core.select.movemap import *

#Protocol Includes
from rosetta.protocols import minimization_packing as pack_min
from rosetta.protocols import relax as rel
from rosetta.protocols.antibody.residue_selector import CDRResidueSelector
from rosetta.protocols.antibody import *
from rosetta.protocols.loops import *
from rosetta.protocols.relax import FastRelax


## Intitlialization 

Here, we will use command-line options to set the relax rounds to 2 instead of default 5 for speed of demo.  This is a bit tricky to do in code.
We also set the input antibody numbering scheme so that Rosetta understands the nomenclature of our antibody. 


Note that typically, we would add these options: `-ex1` and `-ex2` in order to increase the amount of rotamers
 available for packing, but this will slow us down for the demo, so we are keeping this out.

In [2]:
init('-use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res \
     -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2 -no_fconfig')

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Reading fconfig.../Users/jadolfbr/.rosetta/flags/common
core.init: 
core.init: 
core.init: Rosetta version: PyRosetta4.Release.python27.mac r206 2019.01+release.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 http://www.pyrosetta.org 2019-01-03T10:31:13
core.init: command: PyRosetta -use_input_sc -input_ab_scheme AHo_Scheme -ignore_unrecognized_res -ignore_zero_occupancy false -load_PDB_components false -relax:default_repeats 2 -database /Library/Python/2.7/site-packages/pyrosetta-2019.1+release.dbc838b6ae6-py2.7-macosx-10.13-intel.egg/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-153595891 seed_offset=0 real_seed=-153595891
core.init.random: RandomGenerator:init: Normal mode, seed=-153595891 RG_type=mt19937


## Import and copy pose

Begin by importing a pose.  Here, we are going to use an antibody from the PDB. 
Note that we use the clone function to copy the pose into `original_pose`. Using the equal sign will only make what is known as a shallow copy in python and anything we do to pose will be seen in original_pose.  The clone operation copies all the data into the original_pose and is the Rosetta equivalent of the python module, `copy.deep_copy`. 

In [3]:
#Import a pose
pose = pose_from_pdb("inputs/2r0l_1_1.pdb")
original_pose = pose.clone()

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 636 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.76167 seconds.
core.import_pose.import_pose: File '2r0l_1_1.pdb' automatically determined to be of type PDB
core.io.pdb.pdb_reader: Parsing 0 .pdb records with unknown format to search for Rosetta-specific comments.
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 108
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 225
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue ARG:CtermProteinFull 464
core.conformation.Conformation: Found disulfide between residues 23 88
core.conformation.Conformation: current variant for 23 CYS
core.conformation.Conformation: current variant for 88 CYS
core.conformation.Conformation: current variant for 23 CYD
core.conformation.Conformation: current varian

<!--NAVIGATION-->
< [Side Chain Conformations and Dunbrack Energies](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.01-Side-Chain-Conformations-and-Dunbrack-Energies.ipynb) | [Contents](toc.ipynb) | [Index](index.ipynb) | [Protein Design with a Resfile and FastRelax](http://nbviewer.jupyter.org/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.03-Design-with-a-resfile-and-relax.ipynb) ><p><a href="https://colab.research.google.com/github/RosettaCommons/PyRosetta.notebooks/blob/master/notebooks/06.02-Packing-design-and-regional-relax.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>